<a href="https://colab.research.google.com/github/FujunhaoFc/Word2Vec/blob/main/Word2vecPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2 Word Vectors

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data from files
train = pd.read_csv( "word2vec/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "word2vec/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "word2vec/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [3]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [5]:
unlabeled_train.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [6]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [7]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [9]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')
print(nltk.data.path)
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /Users/junhaofu/nltk_data...


['/Users/junhaofu/nltk_data', '/Users/junhaofu/ENTER/nltk_data', '/Users/junhaofu/ENTER/share/nltk_data', '/Users/junhaofu/ENTER/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/var/folders/z5/5fqf9f4x2f9ddt98nd465jh00000gn/T/ipykernel_38956/2861317490.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/var/folders/z5/5fqf9f4x2f9ddt98nd465jh00000gn/T/ipykernel_38956/2861317490.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Parsing sentences from unlabeled set


In [11]:
sentences[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

In [12]:
sentences[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

In [13]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 25.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [14]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2025-10-15 20:10:44,030 : INFO : collecting all words and their counts
2025-10-15 20:10:44,030 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-10-15 20:10:44,076 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types
2025-10-15 20:10:44,108 : INFO : PROGRESS: at sentence #20000, processed 451738 words, keeping 24945 word types
2025-10-15 20:10:44,137 : INFO : PROGRESS: at sentence #30000, processed 670859 words, keeping 30027 word types
2025-10-15 20:10:44,168 : INFO : PROGRESS: at sentence #40000, processed 896841 words, keeping 34335 word types
2025-10-15 20:10:44,202 : INFO : PROGRESS: at sentence #50000, processed 1116082 words, keeping 37751 word types
2025-10-15 20:10:44,239 : INFO : PROGRESS: at sentence #60000, processed 1337544 words, keeping 40711 word types


Training model...


2025-10-15 20:10:44,271 : INFO : PROGRESS: at sentence #70000, processed 1560307 words, keeping 43311 word types
2025-10-15 20:10:44,311 : INFO : PROGRESS: at sentence #80000, processed 1779516 words, keeping 45707 word types
2025-10-15 20:10:44,353 : INFO : PROGRESS: at sentence #90000, processed 2003714 words, keeping 48121 word types
2025-10-15 20:10:44,384 : INFO : PROGRESS: at sentence #100000, processed 2225465 words, keeping 50190 word types
2025-10-15 20:10:44,417 : INFO : PROGRESS: at sentence #110000, processed 2444323 words, keeping 52058 word types
2025-10-15 20:10:44,457 : INFO : PROGRESS: at sentence #120000, processed 2666488 words, keeping 54098 word types
2025-10-15 20:10:44,493 : INFO : PROGRESS: at sentence #130000, processed 2892315 words, keeping 55837 word types
2025-10-15 20:10:44,525 : INFO : PROGRESS: at sentence #140000, processed 3104796 words, keeping 57324 word types
2025-10-15 20:10:44,557 : INFO : PROGRESS: at sentence #150000, processed 3330432 words, ke

In [15]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [16]:
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [17]:
model.wv.doesnt_match("paris berlin london austria".split())

'austria'

In [18]:
model.wv.most_similar("man")

[('woman', 0.6145219802856445),
 ('lad', 0.5886115431785583),
 ('lady', 0.5732795000076294),
 ('millionaire', 0.5231574773788452),
 ('guy', 0.5225004553794861),
 ('monk', 0.5199234485626221),
 ('businessman', 0.5100327730178833),
 ('farmer', 0.5015870928764343),
 ('soldier', 0.5012238025665283),
 ('men', 0.5008193254470825)]

In [19]:
model.wv.most_similar("queen")

[('princess', 0.6551982164382935),
 ('victoria', 0.6020163297653198),
 ('bride', 0.5963108539581299),
 ('belle', 0.5918349623680115),
 ('mistress', 0.5826388001441956),
 ('maid', 0.582522988319397),
 ('showgirl', 0.5799010396003723),
 ('stepmother', 0.5652430057525635),
 ('latifah', 0.5630404949188232),
 ('prince', 0.5611833333969116)]

In [20]:
model.wv.most_similar("awful")

[('terrible', 0.7683419585227966),
 ('atrocious', 0.7360451221466064),
 ('horrible', 0.7290331125259399),
 ('dreadful', 0.6988271474838257),
 ('abysmal', 0.6955971121788025),
 ('horrendous', 0.6760333776473999),
 ('appalling', 0.675796627998352),
 ('horrid', 0.6715326905250549),
 ('amateurish', 0.6361018419265747),
 ('lousy', 0.6212121248245239)]